In [ ]:
import numpy as np
import pandas as pd

In [ ]:
import statsmodels.formula.api as smf
from statsmodels.formula.api import ols
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
# A/B Test 1

In [ ]:
# A/B Test 2

In [ ]:
# Bootstrap

In [ ]:
# Confidence Intervals

In [ ]:
# Correction Analysis

In [ ]:
# Data Visualizations